<a href="https://colab.research.google.com/github/AKTasos/Kaggle-Digit-Recognizer/blob/Conv2dSequential/Google_colav_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
import torch
import math
import time
from collections import namedtuple
from itertools import product
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import json
import torch.nn as nn

In [3]:
path = os.chdir("/content/drive/My Drive/kaggle digit/Kaggle Digit Recognizer")
print(os.listdir())
data_path = str


class TensorDataset(Dataset):
    
    def __init__(self, data_path) : 
        data = pd.read_csv(data_path)
        # data = np.loadtxt("./input/data.csv", delimiter=',', dtype=np.float32, skiprows=1)
        try :
            self.y = torch.from_numpy(data.label.values)
        except :
            self.y = None
        self.x = torch.from_numpy(data.loc[:,data.columns != "label"].values).unsqueeze(1).view(data.shape[0], 1, 28, 28)
        self.n_samples=data.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

['__pycache__', 'input', 'trained_models', 'runs', '.git', 'run_options.py', 'Kaggle_digit_recogniser.py', 'networkfc.py', 'network_fc_template', 'kaggle_submission.csv', 'analytics.py', 'networkconv.py', 'results.json', 'results.csv', 'README.md', 'tensorfromdata.py', 'dataloader.py', 'test.py']


In [0]:
class Run():
    
    def __init__(self):
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        self.run_duration = None
        self.network = None
        self.data_loader = None
        self.tb = None
        
    #params = dictionary of parameters for DataLoader and optim
    # dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn, pin_memory, drop_last, timeout, worker_init_fn)
    
    def run_parameters(self, params):
        Run = namedtuple('Run', params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs
    
    def run_begin(self, run, network, data_loader):
        
        self.run_count += 1
        self.run_start_time = time.time()
        self.run_params = run
        self.tb = SummaryWriter(comment=f'-{run}')
        self.network = network
        self.data_loader = data_loader
        
        # images, labels = next(iter(self.data_loader))
        # grid = torchvision.utils.make_grid(images)
    
        # self.tb.add_image('images', grid)
        # self.tb.add_graph(self.network, images)
    
    def save(self, fileName):
    
        pd.DataFrame.from_dict( self.run_data, orient='columns').to_csv(f'{fileName}.csv')
    
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        PATH = f'./trained_models/{self.run_params}.pth'
        torch.save(self.network.state_dict(), PATH)
    
class Epochs(Run):
    
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        self.epoch_end_time = None
        self.epoch_duration = None
        
        
        
    def start_epoch(self):
        
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        
    def end_epoch(self, r):
        
        self.epoch_duration = time.time() - self.epoch_start_time
        r.run_duration = time.time() - r.run_start_time
    
        loss = self.epoch_loss / len(r.data_loader.dataset)
        accuracy = self.epoch_num_correct / len(r.data_loader.dataset)
        
        
        
        r.tb.add_scalar('Loss', loss, self.epoch_count)
        r.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
    
        # for name, param in r.network.named_parameters():
        #     r.tb.add_histogram(name, param, self.epoch_count)
        #     r.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
            
        results = dict()
        results["run"] = r.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = self.epoch_duration
        results['run duration'] = r.run_duration
        for k,v in r.run_params._asdict().items(): results[k] = v
        r.run_data.append(results)
        
        df = pd.DataFrame.from_dict(r.run_data, orient='columns')
      
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
        
    def track_loss(self, loss, batch_size):
        self.epoch_loss += loss.item() * batch_size
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += correct(preds, labels)
        
        
        
class FcLayers():
    
    def __init__(self, n_in, output, nb_of_layers, act_fonction):
        
        self.n_layer = 1
        self.name = (f'fc{self.n_layer}')
        self.in_features = n_in
        self.n_delta = n_in // nb_of_layers
        self.out_features = n_in-self.n_delta
        self.output = output
        self.nb_of_layers = nb_of_layers
        self.layer_list = []
        self.act_fonction="relu"
        
    def next_layer_parameters(self):
        
        self.n_layer += 1
        self.name = (f'fc{self.n_layer}')
        self.in_features = self.out_features
        self.out_features -= self.n_delta
        
        
    def layer_creation(self):
        for i in range(self.nb_of_layers):
            if i == self.nb_of_layers-1 :
                self.name = "out"
                self.out_features = self.output
            self.layer_list.append((self.name, self.in_features, self.out_features))
            self.next_layer_parameters()
            
            
    def network_creator(self):
        
        text_layer = str()
        text_forward =str()
        l=1
        
        for name, in_feat, out_feat in self.layer_list :
            
            text_layer =text_layer + f'self.{name} = nn.Linear(in_features= {in_feat}, out_features={out_feat})\n        '
            text_forward = text_forward + f'#layer {l} \n        x = self.{name}(x) \n        x = F.{self.act_fonction}(x)\n\n        '
            # layer_list.append(f'self.{name} = nn.Linear(in_features= {in_feat}, out_features={out_feat}')
            # text_layer = '\n'.join(layer_list)
            l += 1
        
        file = open("network_fc_template", "r")
        text = file.read()
        text = text.replace('layers_line', text_layer)
        text = text.replace('forward_line', text_forward)
        file = open(f"networkfc.py", "w")
        file.write(text)
        file.close()
       

In [0]:
def correct(preds, labels):
    c=torch.eq(preds.argmax(dim=1),labels,out=torch.tensor([]))
    return c.sum().item()

def get_all_preds(model, loader):
        all_preds = torch.tensor([])
        for batch in loader:
            images, labels = batch
            preds = model(images.float())
            all_preds = torch.cat((all_preds, preds),dim=0)
            return all_preds
  
def confusion_matrix():
    with torch.no_grad():    
        prediction_loader = DataLoader(dataset=dataset,batch_size=len(dataset),shuffle=False)
        train_preds = get_all_preds(network, prediction_loader)
    
    preds_labels = torch.stack((dataset.y, train_preds.argmax(dim=1)),dim=1)
        
    
    conf_matrix = torch.zeros(10,10,dtype=torch.int32)
    for i in preds_labels:
        l, p = i.tolist()
        conf_matrix[l,p] = conf_matrix[l,p] + 1

In [0]:
class FullConNetwork(nn.Module):
    def __init__(self, images, kernel_size, nb_of_fclayers, out_feat):
        
        self.in_channels = images.shape[1]
        self.img_size = images.shape[2]*images.shape[3]
        self.new_feat_map = self.in_channels*2
        self.kernel_size = kernel_size
        self.layer_output = images.shape[2]
        self.n_delta = None
        self.out_features = None
        self.output = out_feat
        self.nb_of_fclayers = nb_of_fclayers
        
        # super function. It inherits from nn.Module and we can access everythink in nn.Module
        super(FullConNetwork,self).__init__()
        # Function.
    # determine number of convolution according to size of image
        self.nb_of_conv = int(math.log(self.img_size)/math.log(2))
        
    # creating layers
        
        self.cnn = []
        for n in range(self.nb_of_conv) :
            self.cnn += [nn.Conv2d(self.in_channels, self.new_feat_map, self.kernel_size),
                    nn.BatchNorm2d(self.new_feat_map),
                    nn.LeakyReLU()]
            
            self.in_channels = self.new_feat_map
            self.new_feat_map = self.new_feat_map*2
            self.layer_output = (self.layer_output-self.kernel_size)+1
            print(self.layer_output)
        
        self.cnn = nn.Sequential(*self.cnn)
        
        self.n_delta = self.layer_output // nb_of_fclayers
        self.out_features = self.layer_output - self.n_delta
        
        
        self.fc = []
        for n in range(self.nb_of_fclayers) :
            if n == self.nb_of_fclayers-1 :
                self.out_features = self.output
            self.fc += [nn.Linear(in_features = self.layer_output, out_features = self.out_features)
                        , nn.LeakyReLU()]     
            self.layer_output -= self.n_delta
            self.out_features -= self.n_delta
            
        self.fc = nn.Sequential(*self.fc)



    
    def forward(self,x):
        x = self.cnn(x)
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.fc(x)
        print(x.shape)
        return x
    
    


In [9]:

out_feat = 10



#parameters = dictionary of parameters for DataLoader and optim (dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn, pin_memory, drop_last, timeout, worker_init_fn)
parameters = dict(
    lr = [0.01]
    ,batch_size = [1000]
    ,shuffle = [False]
    ,epochs = [20]
    ,nb_of_fclayers = [x for x in range(4, 20, 4)]
    # ,act_fonction=["relu","glu","tanh","sigmoid","softmax"]
    ,kernel_size = [2, 4])

r = Run()
runs = r.run_parameters(parameters)

data_path = "input/train.csv"
train_set = TensorDataset(data_path)



for run in runs:
    print(run)
  
    data_loader = DataLoader(dataset=train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    batch = next(iter(data_loader))
    images, labels = batch
    
    cnnfc = FullConNetwork(images, run.kernel_size, run.nb_of_fclayers, out_feat)
    
    optimizer = optim.SGD(cnnfc.parameters(), lr=run.lr)
    
    r.run_begin(run, cnnfc, data_loader)
   
    e = Epochs()
    for epoch in range(run.epochs):
          
        e.start_epoch()
        
        for batch in data_loader :
            
            images, labels = batch
            
            #runs the batch in the CNN
            preds=cnnfc(images.float())
            
            
            #calculate Loss
            loss = F.cross_entropy(preds,labels)
            optimizer.zero_grad()
            
            #BackProp
            loss.backward()
            
            #update weights
            optimizer.step()
        
            e.track_loss(loss, run.batch_size)
            e.track_num_correct(preds, labels)
            
            
        print("epoch:", e.epoch_count ,"/  total_correct:", e.epoch_num_correct, "/  Loss:", e.epoch_loss )
        e.end_epoch(r)
        
    
    r.end_run()
    r.save('results')

Run(lr=0.01, batch_size=1000, shuffle=False, epochs=20, nb_of_fclayers=4, kernel_size=2)
27
26
25
24
23
22
21
20
19
tensor([[[[-8.2240e-03, -8.2240e-03, -8.2240e-03,  ..., -8.7054e-03,
           -1.0404e-02,  6.6918e-01],
          [-8.2240e-03, -8.2240e-03, -8.2240e-03,  ..., -7.3460e-03,
           -2.6671e-03,  5.0276e-01],
          [-8.2240e-03, -8.2240e-03, -8.2240e-03,  ..., -7.5929e-03,
           -9.1682e-03,  1.0586e+00],
          ...,
          [ 4.3008e-01,  1.1710e+00,  1.6175e+00,  ..., -8.2240e-03,
           -8.2240e-03, -8.2240e-03],
          [ 9.1403e-01,  1.5525e+00, -2.6025e-03,  ..., -8.2240e-03,
           -8.2240e-03, -8.2240e-03],
          [ 1.0603e-01,  2.2050e+00,  9.3822e-01,  ..., -8.2240e-03,
           -8.2240e-03, -8.2240e-03]],

         [[-1.2969e-03, -1.2969e-03, -1.2969e-03,  ..., -7.7300e-03,
            8.0276e-02,  3.4623e-01],
          [-1.2969e-03, -1.2969e-03, -1.2969e-03,  ..., -9.2311e-03,
           -8.3132e-04, -1.2517e-02],
          [

RuntimeError: ignored